In [219]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
#import lightgbm as lgb
import datetime
import math
import gc

# Creating new features 

In [104]:
train = pd.read_csv("df_train_week2_update_isrc_name.csv")

/Users/yanzeng/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [105]:
test = pd.read_csv("df_test_week2_update_isrc_name.csv")

/Users/yanzeng/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [206]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

### getting to know the training data set

In [106]:
train.columns

Index(['msno', 'song_id', 'source_system_tab', 'source_screen_name',
       'source_type', 'city', 'bd', 'gender', 'registered_via',
       'registration_init_time', 'expiration_date', 'song_length', 'genre_ids',
       'artist_name', 'composer', 'lyricist', 'language', 'name', 'isrc',
       'is_train'],
      dtype='object')

In [ ]:
# categorical features:
'source_system_tab', 'source_screen_name',
       'source_type', 'city', 'bd', 'gender', 'registered_via',
      ,,  'language', 'is_train', 'isrc'

In [ ]:
# other features:
 'registration_init_time', 'expiration_date'
    'genre_ids',
       'artist_name', 'composer', 'lyricist','name'

In [ ]:
# numercial features:
 'song_length'

In [44]:
train.isnull().sum()

msno                           0
song_id                        0
source_system_tab              0
source_screen_name             0
source_type                    0
city                           0
bd                             0
gender                         0
registered_via                 0
registration_init_time         0
expiration_date                0
song_length                    0
genre_ids                      0
artist_name                    0
composer                       0
lyricist                       0
language                       0
name                        1571
isrc                      577963
is_train                       0
dtype: int64

In [46]:
test.isnull().sum()

msno                           0
song_id                        0
source_system_tab              0
source_screen_name             0
source_type                    0
city                           0
bd                             0
gender                         0
registered_via                 0
registration_init_time         0
expiration_date                0
song_length                    0
genre_ids                      0
artist_name                    0
composer                       0
lyricist                       0
language                       0
name                         803
isrc                      196665
is_train                       0
dtype: int64

In [107]:
song2 = pd.read_csv("song_extra_info.csv")

In [108]:
song  = pd.read_csv("songs.csv")

In [109]:
train = pd.merge(train, song2, how='left', on=['song_id'])
test = pd.merge(test, song2, how='left', on=['song_id'])

In [110]:
train.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,city,bd,gender,registered_via,registration_init_time,...,genre_ids,artist_name,composer,lyricist,language,name_x,isrc_x,is_train,name_y,isrc_y
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1.0,0.0,unknown,7.0,20120102.0,...,359,Bastille,Dan Smith| Mark Crew,no_lyricist,52.0,Good Grief,GBUM71602854,1,Good Grief,GBUM71602854
1,pouJqjNRmZOnRNzzMWWkamTKkIGHyvhl/jo4HgbncnM=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,discover,Online playlist more,online-playlist,15.0,18.0,male,4.0,20151220.0,...,359,Bastille,Dan Smith| Mark Crew,no_lyricist,52.0,Good Grief,GBUM71602854,1,Good Grief,GBUM71602854
2,xbodnNBaLMyqqI7uFJlvHOKMJaizuWo/BB/YHZICcKo=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,my library,Local playlist more,local-library,1.0,0.0,unknown,7.0,20120804.0,...,359,Bastille,Dan Smith| Mark Crew,no_lyricist,52.0,Good Grief,GBUM71602854,1,Good Grief,GBUM71602854
3,s0ndDsjI79amU0RBiullFN8HRz9HjE++34jGNa7zJ/s=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,my library,Local playlist more,local-library,5.0,21.0,female,9.0,20110808.0,...,359,Bastille,Dan Smith| Mark Crew,no_lyricist,52.0,Good Grief,GBUM71602854,1,Good Grief,GBUM71602854
4,Vw4Umh6/qlsJDC/XMslyAxVvRgFJGHr53yb/nrmY1DU=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,my library,Local playlist more,local-library,6.0,33.0,female,9.0,20070323.0,...,359,Bastille,Dan Smith| Mark Crew,no_lyricist,52.0,Good Grief,GBUM71602854,1,Good Grief,GBUM71602854


In [111]:
train.name_x.fillna(0, inplace = True)
train.isrc_x.fillna(0, inplace = True)
test.name_x.fillna(0, inplace = True)
test.isrc_x.fillna(0, inplace = True)

In [112]:
train.name_y.fillna(0, inplace = True)
train.isrc_y.fillna(0, inplace = True)
test.name_y.fillna(0, inplace = True)
test.isrc_y.fillna(0, inplace = True)

In [113]:
train.loc[((train.name_x == 0)&(train.name_y != 0)), 'name_x'] = train.name_y
train.loc[((train.isrc_x == 0)&(train.isrc_y != 0)), 'isrc_x'] = train.isrc_y
test.loc[((test.name_x == 0)&(test.name_y != 0)), 'name_x'] = test.name_y
test.loc[((test.isrc_x == 0)&(test.isrc_y != 0)), 'isrc_x'] = test.isrc_y

In [114]:
train.loc[(train.name_x == 0)].song_id.nunique()

160

In [115]:
train = train.drop(['name_y', 'isrc_y'], axis =1)
test = test.drop(['name_y', 'isrc_y'], axis =1)

In [116]:
song = song[['song_id','name','isrc']]

In [117]:
train = pd.merge(train, song, how='left', on=['song_id'])
test = pd.merge(test, song, how='left', on=['song_id'])

In [118]:
train.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,city,bd,gender,registered_via,registration_init_time,...,genre_ids,artist_name,composer,lyricist,language,name_x,isrc_x,is_train,name,isrc
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1.0,0.0,unknown,7.0,20120102.0,...,359,Bastille,Dan Smith| Mark Crew,no_lyricist,52.0,Good Grief,GBUM71602854,1,Good Grief,GBUM71602854
1,pouJqjNRmZOnRNzzMWWkamTKkIGHyvhl/jo4HgbncnM=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,discover,Online playlist more,online-playlist,15.0,18.0,male,4.0,20151220.0,...,359,Bastille,Dan Smith| Mark Crew,no_lyricist,52.0,Good Grief,GBUM71602854,1,Good Grief,GBUM71602854
2,xbodnNBaLMyqqI7uFJlvHOKMJaizuWo/BB/YHZICcKo=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,my library,Local playlist more,local-library,1.0,0.0,unknown,7.0,20120804.0,...,359,Bastille,Dan Smith| Mark Crew,no_lyricist,52.0,Good Grief,GBUM71602854,1,Good Grief,GBUM71602854
3,s0ndDsjI79amU0RBiullFN8HRz9HjE++34jGNa7zJ/s=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,my library,Local playlist more,local-library,5.0,21.0,female,9.0,20110808.0,...,359,Bastille,Dan Smith| Mark Crew,no_lyricist,52.0,Good Grief,GBUM71602854,1,Good Grief,GBUM71602854
4,Vw4Umh6/qlsJDC/XMslyAxVvRgFJGHr53yb/nrmY1DU=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,my library,Local playlist more,local-library,6.0,33.0,female,9.0,20070323.0,...,359,Bastille,Dan Smith| Mark Crew,no_lyricist,52.0,Good Grief,GBUM71602854,1,Good Grief,GBUM71602854


In [119]:
train.name.fillna(0, inplace = True)
train.isrc.fillna(0, inplace = True)
test.name.fillna(0, inplace = True)
test.isrc.fillna(0, inplace = True)

In [120]:
train.loc[((train.name_x == 0)&(train.name != 0)), 'name_x'] = train.name
train.loc[((train.isrc_x == 0)&(train.isrc != 0)), 'isrc_x'] = train.isrc
test.loc[((test.name_x == 0)&(test.name != 0)), 'name_x'] = test.name
test.loc[((test.isrc_x == 0)&(test.isrc != 0)), 'isrc_x'] = test.isrc

In [121]:
train.loc[(train.name_x == 0)].song_id.nunique()

160

In [122]:
train.loc[(train.isrc_x == 0)].song_id.nunique()

48338

In [123]:
train.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,city,bd,gender,registered_via,registration_init_time,...,genre_ids,artist_name,composer,lyricist,language,name_x,isrc_x,is_train,name,isrc
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1.0,0.0,unknown,7.0,20120102.0,...,359,Bastille,Dan Smith| Mark Crew,no_lyricist,52.0,Good Grief,GBUM71602854,1,Good Grief,GBUM71602854
1,pouJqjNRmZOnRNzzMWWkamTKkIGHyvhl/jo4HgbncnM=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,discover,Online playlist more,online-playlist,15.0,18.0,male,4.0,20151220.0,...,359,Bastille,Dan Smith| Mark Crew,no_lyricist,52.0,Good Grief,GBUM71602854,1,Good Grief,GBUM71602854
2,xbodnNBaLMyqqI7uFJlvHOKMJaizuWo/BB/YHZICcKo=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,my library,Local playlist more,local-library,1.0,0.0,unknown,7.0,20120804.0,...,359,Bastille,Dan Smith| Mark Crew,no_lyricist,52.0,Good Grief,GBUM71602854,1,Good Grief,GBUM71602854
3,s0ndDsjI79amU0RBiullFN8HRz9HjE++34jGNa7zJ/s=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,my library,Local playlist more,local-library,5.0,21.0,female,9.0,20110808.0,...,359,Bastille,Dan Smith| Mark Crew,no_lyricist,52.0,Good Grief,GBUM71602854,1,Good Grief,GBUM71602854
4,Vw4Umh6/qlsJDC/XMslyAxVvRgFJGHr53yb/nrmY1DU=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,my library,Local playlist more,local-library,6.0,33.0,female,9.0,20070323.0,...,359,Bastille,Dan Smith| Mark Crew,no_lyricist,52.0,Good Grief,GBUM71602854,1,Good Grief,GBUM71602854


In [124]:
train.drop(['name','isrc'], axis = 1, inplace = True)
test.drop(['name','isrc'], axis = 1, inplace = True)

In [125]:
train.rename(columns={'name_x': 'name', 'isrc_x': 'isrc'}, inplace=True)
test.rename(columns={'name_x': 'name', 'isrc_x': 'isrc'}, inplace=True)

In [126]:
train.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,city,bd,gender,registered_via,registration_init_time,expiration_date,song_length,genre_ids,artist_name,composer,lyricist,language,name,isrc,is_train
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1.0,0.0,unknown,7.0,20120102.0,20171005.0,206471.0,359,Bastille,Dan Smith| Mark Crew,no_lyricist,52.0,Good Grief,GBUM71602854,1
1,pouJqjNRmZOnRNzzMWWkamTKkIGHyvhl/jo4HgbncnM=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,discover,Online playlist more,online-playlist,15.0,18.0,male,4.0,20151220.0,20170930.0,206471.0,359,Bastille,Dan Smith| Mark Crew,no_lyricist,52.0,Good Grief,GBUM71602854,1
2,xbodnNBaLMyqqI7uFJlvHOKMJaizuWo/BB/YHZICcKo=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,my library,Local playlist more,local-library,1.0,0.0,unknown,7.0,20120804.0,20171004.0,206471.0,359,Bastille,Dan Smith| Mark Crew,no_lyricist,52.0,Good Grief,GBUM71602854,1
3,s0ndDsjI79amU0RBiullFN8HRz9HjE++34jGNa7zJ/s=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,my library,Local playlist more,local-library,5.0,21.0,female,9.0,20110808.0,20170917.0,206471.0,359,Bastille,Dan Smith| Mark Crew,no_lyricist,52.0,Good Grief,GBUM71602854,1
4,Vw4Umh6/qlsJDC/XMslyAxVvRgFJGHr53yb/nrmY1DU=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,my library,Local playlist more,local-library,6.0,33.0,female,9.0,20070323.0,20170915.0,206471.0,359,Bastille,Dan Smith| Mark Crew,no_lyricist,52.0,Good Grief,GBUM71602854,1


In [18]:
print("num of distinct train's song_id: ", train.song_id.nunique())
print("num of distinct song2's song_id: ", song2.song_id.nunique())

num of distinct train's song_id:  359966
num of distinct song2's song_id:  2295971


In [20]:
print('Song ids present in both: ', len(set(train.song_id) & set(song2.song_id)))

print('Only in train but not in song2: ', len(set(train.song_id) - (set(train.song_id) & set(song2.song_id))))

Song ids present in both:  359807
Only in train but not in song2:  159


In [55]:
train.loc[train.name_x == 0].song_id.nunique()

212

In [54]:
print('Song ids present in both: ', len(set(train.loc[train.name_x == 0].song_id.unique()) & set(song.song_id)))

Song ids present in both:  160


In [50]:
print('Song ids present in both: ', len(set(train.song_id) & set(song.song_id)))

print('Only in train but not in song2: ', len(set(train.song_id) - (set(train.song_id) & set(song.song_id))))

Song ids present in both:  359914
Only in train but not in song2:  52


In [127]:
train.isrc.describe()

count     7377418
unique     269761
top             0
freq       577858
Name: isrc, dtype: int64

In [129]:
test.isrc.describe()

count     2556790
unique     170988
top             0
freq       196643
Name: isrc, dtype: int64

In [130]:
train.loc[train.name == 0, 'name'] = 'unknown'
#train.loc[train.isrc == 0, 'isrc'] = 'unknown'
test.loc[test.name == 0, 'name'] = 'unknown'
#test.loc[test.isrc == 0, 'isrc'] = 'unknown'

In [101]:
train.isnull().sum()

msno                      0
song_id                   0
source_system_tab         0
source_screen_name        0
source_type               0
city                      0
bd                        0
gender                    0
registered_via            0
registration_init_time    0
expiration_date           0
song_length               0
genre_ids                 0
artist_name               0
composer                  0
lyricist                  0
language                  0
name                      0
isrc                      0
is_train                  0
dtype: int64

In [131]:
train.to_csv("yan_week3_imputed_train.csv", index = False)
test.to_csv("yan_week3_imputed_test.csv", index = False)

In [135]:
train.loc[train.isrc == 0].shape

(577858, 21)

### Converting isrc to year feature

In [132]:
def isrc_to_year(isrc):
    if type(isrc) == str:
        if int(isrc[5:7]) > 17:
            return 1900 + int(isrc[5:7])
        else:
            return 2000 + int(isrc[5:7])
    else:
        return np.nan
        
train['song_year'] = train['isrc'].apply(isrc_to_year)
#train.drop(['isrc', 'name'], axis = 1, inplace = True)

test['song_year'] = test['isrc'].apply(isrc_to_year)
#test.drop(['isrc', 'name'], axis = 1, inplace = True)

In [133]:
train.isnull().sum()

msno                           0
song_id                        0
source_system_tab              0
source_screen_name             0
source_type                    0
city                           0
bd                             0
gender                         0
registered_via                 0
registration_init_time         0
expiration_date                0
song_length                    0
genre_ids                      0
artist_name                    0
composer                       0
lyricist                       0
language                       0
name                           0
isrc                           0
is_train                       0
song_year                 577858
dtype: int64

In [136]:
577858/train.shape[0]

0.07832794617303777

In [137]:
train.song_year.fillna(0, inplace = True)
test.song_year.fillna(0, inplace = True)

In [142]:
train.loc[train.song_year == 0, 'song_year'] = train.song_year.mode().values[0]
test.loc[test.song_year == 0, 'song_year'] = test.song_year.mode().values[0]

In [155]:
# columns needed to delete
#train.drop(['isrc', 'name'], axis = 1, inplace = True)
#train = train.drop(['registration_init_time'], axis=1)
#test.drop(['isrc', 'name'], axis = 1, inplace = True)
#test = test.drop(['registration_init_time'], axis=1)

### Converting registration_init_time and expiration_date into features

In [149]:
train['expiration_date'] = pd.to_datetime(train['expiration_date'], format='%Y%m%d')
train['registration_init_time']= pd.to_datetime(train['registration_init_time'], format='%Y%m%d')

In [150]:
train['expiration_date'].head()

0   2017-10-05
1   2017-09-30
2   2017-10-04
3   2017-09-17
4   2017-09-15
Name: expiration_date, dtype: datetime64[ns]

In [151]:
train['membership_days'] = train['expiration_date'].subtract(train['registration_init_time']).dt.days.astype(int)

train['registration_year'] = train['registration_init_time'].dt.year
train['registration_month'] = train['registration_init_time'].dt.month
train['registration_date'] = train['registration_init_time'].dt.day

train['expiration_year'] = train['expiration_date'].dt.year
train['expiration_month'] = train['expiration_date'].dt.month
train['expiration_date'] = train['expiration_date'].dt.day
#train = train.drop(['registration_init_time'], axis=1)

In [153]:
test['expiration_date'] = pd.to_datetime(test['expiration_date'], format='%Y%m%d')
test['registration_init_time']= pd.to_datetime(test['registration_init_time'], format='%Y%m%d')

In [154]:
test['membership_days'] = test['expiration_date'].subtract(test['registration_init_time']).dt.days.astype(int)

test['registration_year'] = test['registration_init_time'].dt.year
test['registration_month'] = test['registration_init_time'].dt.month
test['registration_date'] = test['registration_init_time'].dt.day

test['expiration_year'] = test['expiration_date'].dt.year
test['expiration_month'] = test['expiration_date'].dt.month
test['expiration_date'] = test['expiration_date'].dt.day
#test = test.drop(['registration_init_time'], axis=1)

In [229]:
train['expiration_year'].value_counts()

2017    6426684
2018     744417
2016     184832
2015       9014
2019       6675
2013       1631
2014       1125
2012        750
2020        688
2011        390
2008        388
2007        244
2004        227
2005        133
2006        132
2010         44
2009         41
1970          3
Name: expiration_year, dtype: int64

### Creating count of genre ids

train['genre_ids'] = train['genre_ids'].astype(str)
test['genre_ids'] = test['genre_ids'].astype(str)

In [177]:
def genre_id_count(x):
    if x == 'no_genre_id':
        return 0
    else:
        return x.count('|') + 1
train['genre_ids_count'] = train['genre_ids'].apply(genre_id_count).astype(np.int8)
test['genre_ids_count'] = test['genre_ids'].apply(genre_id_count).astype(np.int8)

In [193]:
train['genre_ids_count'].value_counts()

1    6918646
2     298160
0     118455
3      33682
4       6096
5       2120
8        213
6         33
7         13
Name: genre_ids_count, dtype: int64

### Creating count of lyricists

In [179]:
def lyricist_count(x):
    if x == 'no_lyricist':
        return 0
    else:
        return sum(map(x.count, ['|', '/', '\\', ';'])) + 1
    return sum(map(x.count, ['|', '/', '\\', ';']))


train['lyricists_count'] = train['lyricist'].apply(lyricist_count).astype(np.int8)
test['lyricists_count'] = test['lyricist'].apply(lyricist_count).astype(np.int8)

In [224]:
train['lyricist']

0                                                no_lyricist
1                                                no_lyricist
2                                                no_lyricist
3                                                no_lyricist
4                                                no_lyricist
5                                                no_lyricist
6                                                no_lyricist
7                                                no_lyricist
8                                                no_lyricist
9                                                no_lyricist
10                                               no_lyricist
11                                               no_lyricist
12                                               no_lyricist
13                                               no_lyricist
14                                               no_lyricist
15                                               no_lyricist
16                      

In [192]:
train['lyricists_count'].value_counts()

0     3178798
1     3140124
2      607400
3      198073
4       99820
6       48760
5       48356
7       23640
8       10457
11       8156
9        7781
10       3392
12        861
20        719
16        478
14        477
13         64
15         27
25         13
23          9
18          5
19          4
17          3
21          1
Name: lyricists_count, dtype: int64

### Creating count of composer

In [187]:
def composer_count(x):
    if x == 'no_composer':
        return 0
    else:
        return sum(map(x.count, ['|', '/', '\\', ';'])) + 1

train['composer_count'] = train['composer'].apply(composer_count).astype(np.int8)
test['composer_count'] = test['composer'].apply(composer_count).astype(np.int8)

In [191]:
train['composer_count'].value_counts()

1      3940184
0      1675706
2       673216
3       432070
4       237160
5       178587
6       117604
7        50841
8        19020
9        17405
11       13004
10       12211
12        3438
17        2139
13        1187
16         946
14         903
20         730
19         382
18         378
15         221
27          36
21          15
25          13
22          12
23           9
108          1
Name: composer_count, dtype: int64

### Creating count of artists

In [195]:
train[train['artist_name'] == 'no_artist']

,msno,song_id,source_system_tab,source_screen_name,source_type,city,bd,gender,registered_via,registration_init_time,expiration_date,song_length,genre_ids,artist_name,composer,lyricist,language,name,isrc,is_train,song_year,membership_days,registration_year,registration_month,registration_date,expiration_year,expiration_month,genre_ids_count,lyricists_count,composer_count,is_featured
5841043,n+f0bRfah4zmn4c+ek4wWGoTmwW78SUpi3qk1MoN29U=,WKCEUOm70//9O7IQ3VE2ohtiJaDGUH7cChanz4plSdA=,discover,other_sources,song-based-playlist,1.0,0.0,unknown,7.0,2015-09-05,4,245120.96936,no_genre_id,no_artist,no_composer,no_lyricist,0.0,超級4th場之24hr不打烊情歌,TWA450995811,1,2009.0,730,2015,9,5,2017,9,0,0,0,0
5841044,ITiE2zbCjjXoGflqFUa9pbJT35i3eKao4b2tTfyAShQ=,WKCEUOm70//9O7IQ3VE2ohtiJaDGUH7cChanz4plSdA=,my library,Local playlist more,local-playlist,5.0,24.0,male,7.0,2013-01-19,4,245120.96936,no_genre_id,no_artist,no_composer,no_lyricist,0.0,超級4th場之24hr不打烊情歌,TWA450995811,1,2009.0,1719,2013,1,19,2017,10,0,0,0,0
5841045,BF97sQNyf77oqUSZH1SZHvJLbSUqhk88Zv/7BD1/ed8=,WKCEUOm70//9O7IQ3VE2ohtiJaDGUH7cChanz4plSdA=,my library,Local playlist more,local-playlist,4.0,26.0,male,3.0,2013-08-01,30,245120.96936,no_genre_id,no_artist,no_composer,no_lyricist,0.0,超級4th場之24hr不打烊情歌,TWA450995811,1,2009.0,1521,2013,8,1,2017,9,0,0,0,0
5841046,L1tOhpoNmd8Uva+R8bWlJFBuBVmyJdzG3VaSNs0Khck=,WKCEUOm70//9O7IQ3VE2ohtiJaDGUH7cChanz4plSdA=,my library,Local playlist more,local-library,4.0,24.0,female,9.0,2007-03-17,1,245120.96936,no_genre_id,no_artist,no_composer,no_lyricist,0.0,超級4th場之24hr不打烊情歌,TWA450995811,1,2009.0,3851,2007,3,17,2017,10,0,0,0,0
5841047,vd9pM2k78xANW3GQsFS3S6Xr5UMNpdwBQPlKNyz2x3k=,WKCEUOm70//9O7IQ3VE2ohtiJaDGUH7cChanz4plSdA=,my library,Local playlist more,local-playlist,5.0,0.0,unknown,9.0,2012-10-21,19,245120.96936,no_genre_id,no_artist,no_composer,no_lyricist,0.0,超級4th場之24hr不打烊情歌,TWA450995811,1,2009.0,1794,2012,10,21,2017,9,0,0,0,0
5841048,weyBPJj+ionjF9ryHb3S7RS3N6+X6RNvmrclxypQ0/8=,WKCEUOm70//9O7IQ3VE2ohtiJaDGUH7cChanz4plSdA=,search,Search,song,13.0,32.0,male,9.0,2004-09-02,30,245120.96936,no_genre_id,no_artist,no_composer,no_lyricist,0.0,超級4th場之24hr不打烊情歌,TWA450995811,1,2009.0,4776,2004,9,2,2017,9,0,0,0,0
5841049,80XCaFoSB4bHzQ3vdB/OFOPya2iI7YdcVR5DNUro1rY=,WKCEUOm70//9O7IQ3VE2ohtiJaDGUH7cChanz4plSdA=,my library,Local playlist more,local-library,14.0,23.0,female,7.0,2014-10-19,11,245120.96936,no_genre_id,no_artist,no_composer,no_lyricist,0.0,超級4th場之24hr不打烊情歌,TWA450995811,1,2009.0,1058,2014,10,19,2017,9,0,0,0,0
5841050,Kd6SIN/eErf+MywBAn1jA/SOtVdnEMNE0Ofs4J/hFcE=,WKCEUOm70//9O7IQ3VE2ohtiJaDGUH7cChanz4plSdA=,my library,Local playlist more,local-playlist,12.0,28.0,male,7.0,2015-12-06,2,245120.96936,no_genre_id,no_artist,no_composer,no_lyricist,0.0,超級4th場之24hr不打烊情歌,TWA450995811,1,2009.0,666,2015,12,6,2017,10,0,0,0,0
5841051,DBrDJMl0bOsVck3g6EO1AZ/GgK2vMPwU8Ob/lTu8We4=,WKCEUOm70//9O7IQ3VE2ohtiJaDGUH7cChanz4plSdA=,my library,Local playlist more,local-library,22.0,30.0,female,7.0,2013-09-07,22,245120.96936,no_genre_id,no_artist,no_composer,no_lyricist,0.0,超級4th場之24hr不打烊情歌,TWA450995811,1,2009.0,1476,2013,9,7,2017,9,0,0,0,0
5841052,pjuHeDvlMLop4cFg3eAOxTfjzpKeW1jhMfYbi2YAUQI=,WKCEUOm70//9O7IQ3VE2ohtiJaDGUH7cChanz4plSdA=,my library,Local playlist more,local-library,1.0,20.0,male,3.0,2015-07-17,9,245120.96936,no_genre_id,no_artist,no_composer,no_lyricist,0.0,超級4th場之24hr不打烊情歌,TWA450995811,1,2009.0,632,2015,7,17,2017,4,0,0,0,0


In [196]:
def artist_count(x):
    if x == 'no_artist':
        return 0
    else:
        return x.count('and') + x.count(',') + x.count('feat') + x.count('&')

train['artist_count'] = train['artist_name'].apply(artist_count).astype(np.int8)
test['artist_count'] = test['artist_name'].apply(artist_count).astype(np.int8)

In [198]:
train['artist_count'].value_counts()

0     7171826
1      191852
2       12790
3         398
8         335
4         136
6          47
5          26
12          5
7           3
Name: artist_count, dtype: int64

### whether the song is shorter than average song length

In [205]:
train['song_length'].unique()

array([ 206471.,  187802.,  247803., ...,  101929.,  405943.,  105439.])

In [207]:
_mean_song_length = np.mean(train['song_length'])
def smaller_song(x):
    if x < _mean_song_length:
        return 1
    return 0

train['smaller_song'] = train['song_length'].apply(smaller_song).astype(np.int8)
test['smaller_song'] = test['song_length'].apply(smaller_song).astype(np.int8)

In [208]:
train['smaller_song'].value_counts()

1    3896801
0    3480617
Name: smaller_song, dtype: int64

### num times a song has been played

In [209]:
_dict_count_song_played_train = {k: v for k, v in train['song_id'].value_counts().iteritems()}
_dict_count_song_played_test = {k: v for k, v in test['song_id'].value_counts().iteritems()}
def count_song_played(x):
    try:
        return _dict_count_song_played_train[x]
    except KeyError:
        try:
            return _dict_count_song_played_test[x]
        except KeyError:
            return 0
    

train['count_song_played'] = train['song_id'].apply(count_song_played).astype(np.int64)
test['count_song_played'] = test['song_id'].apply(count_song_played).astype(np.int64)

### num times the artist has been played

In [215]:
_dict_count_artist_played_train = {k: v for k, v in train['artist_name'].value_counts().iteritems()}
_dict_count_artist_played_test = {k: v for k, v in test['artist_name'].value_counts().iteritems()}
def count_artist_played(x):
    try:
        return _dict_count_artist_played_train[x]
    except KeyError:
        try:
            return _dict_count_artist_played_test[x]
        except KeyError:
            return 0

train['count_artist_played'] = train['artist_name'].apply(count_artist_played).astype(np.int64)
test['count_artist_played'] = test['artist_name'].apply(count_artist_played).astype(np.int64)

In [216]:
train.to_csv("newest_1_train.csv", index = False)
test.to_csv("newest_1_test.csv", index = False)

In [217]:
train.shape


(7377418, 37)

In [218]:
test.shape

(2556790, 37)

In [220]:
# Wednesday continue
train = pd.read_csv("newest_1_train.csv")
test = pd.read_csv("newest_1_test.csv")

/Users/yanzeng/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### For final training models

In [ ]:
#print ("Train test and validation sets")
#for col in train.columns:
#    if train[col].dtype == object:
#        train[col] = train[col].astype('category')
#        test[col] = test[col].astype('category')


# X_train = train.drop(['target'], axis=1)
# y_train = train['target'].values


# X_test = test.drop(['id'], axis=1)
# ids = test['id'].values

In [221]:
train.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,city,bd,gender,registered_via,registration_init_time,expiration_date,song_length,genre_ids,artist_name,composer,lyricist,language,name,isrc,is_train,song_year,membership_days,registration_year,registration_month,registration_date,expiration_year,expiration_month,genre_ids_count,lyricists_count,composer_count,is_featured,artist_count,artist_composer,artist_composer_lyricist,smaller_song,count_song_played,count_artist_played
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1.0,0.0,unknown,7.0,2012-01-02,5,206471.0,359,Bastille,Dan Smith| Mark Crew,no_lyricist,52.0,Good Grief,GBUM71602854,1,2016.0,2103,2012,1,2,2017,10,1,0,2,0,0,0,0,1,215,1140
1,pouJqjNRmZOnRNzzMWWkamTKkIGHyvhl/jo4HgbncnM=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,discover,Online playlist more,online-playlist,15.0,18.0,male,4.0,2015-12-20,30,206471.0,359,Bastille,Dan Smith| Mark Crew,no_lyricist,52.0,Good Grief,GBUM71602854,1,2016.0,650,2015,12,20,2017,9,1,0,2,0,0,0,0,1,215,1140
2,xbodnNBaLMyqqI7uFJlvHOKMJaizuWo/BB/YHZICcKo=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,my library,Local playlist more,local-library,1.0,0.0,unknown,7.0,2012-08-04,4,206471.0,359,Bastille,Dan Smith| Mark Crew,no_lyricist,52.0,Good Grief,GBUM71602854,1,2016.0,1887,2012,8,4,2017,10,1,0,2,0,0,0,0,1,215,1140
3,s0ndDsjI79amU0RBiullFN8HRz9HjE++34jGNa7zJ/s=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,my library,Local playlist more,local-library,5.0,21.0,female,9.0,2011-08-08,17,206471.0,359,Bastille,Dan Smith| Mark Crew,no_lyricist,52.0,Good Grief,GBUM71602854,1,2016.0,2232,2011,8,8,2017,9,1,0,2,0,0,0,0,1,215,1140
4,Vw4Umh6/qlsJDC/XMslyAxVvRgFJGHr53yb/nrmY1DU=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,my library,Local playlist more,local-library,6.0,33.0,female,9.0,2007-03-23,15,206471.0,359,Bastille,Dan Smith| Mark Crew,no_lyricist,52.0,Good Grief,GBUM71602854,1,2016.0,3829,2007,3,23,2017,9,1,0,2,0,0,0,0,1,215,1140


In [223]:
train['count_song_played'].max()

13973

### num of songs each artist has

In [278]:
_dict_num_songs_per_artist_train = {k: v for k, v in train.groupby('artist_name')['song_id'].nunique().iteritems()}
_dict_num_songs_per_artist_test = {k: v for k, v in test.groupby('artist_name')['song_id'].nunique().iteritems()}
def count_songs_for_artist(x):
    try:
        return _dict_num_songs_per_artist_train[x]
    except KeyError:
        try:
            return _dict_num_songs_per_artist_test[x]
        except KeyError:
            return 0

train['num_songs_for_artist'] = train['artist_name'].apply(count_songs_for_artist).astype(np.int64)
test['num_songs_for_artist'] = test['artist_name'].apply(count_songs_for_artist).astype(np.int64)

In [280]:
# double check: train2[train2.artist_name == 'Bastille']['song_id'].nunique()
# train2[train2.artist_name == '100 Best Adagio Voices']['song_id'].nunique()

### num languages the artist sings in

In [288]:
_dict_num_languages_per_artist_train = {k: v for k, v in train.groupby('artist_name')['language'].nunique().iteritems()}
_dict_num_languages_per_artist_test = {k: v for k, v in test.groupby('artist_name')['language'].nunique().iteritems()}
def num_languages_for_artist(x):
    try:
        return _dict_num_languages_per_artist_train[x]
    except KeyError:
        try:
            return _dict_num_languages_per_artist_test[x]
        except KeyError:
            return 0

train['num_languages_for_artist'] = train['artist_name'].apply(num_languages_for_artist).astype(np.int64)
test['num_languages_for_artist'] = test['artist_name'].apply(num_languages_for_artist).astype(np.int64)

In [293]:
#train[train.artist_name == 'Bastille'].num_languages_for_artist
#train[train.artist_name == 'Bastille']['language'].nunique()

## Prove that no duplicates exist

In [234]:
train1 = train.copy()[['msno', 'song_id']]
test1 = test.copy()[['msno', 'song_id']]

In [235]:
train1.shape

(7377418, 2)

In [238]:
train1 = train1.drop_duplicates()

In [239]:
train1.shape

(7377418, 2)

###  whether the featured artist has performed or not

In [189]:
def is_featured(x):
    if 'feat' in str(x):
        return 1
    else:
        return 0

train['is_featured'] = train['artist_name'].apply(is_featured).astype(np.int8)
test['is_featured'] = test['artist_name'].apply(is_featured).astype(np.int8)

In [190]:
train['is_featured'].value_counts()

0    7362177
1      15241
Name: is_featured, dtype: int64

### artist_composer: whether the same

In [197]:
train['artist_composer'] = (train['artist_name'] == train['composer']).astype(np.int8)
test['artist_composer'] = (test['artist_name'] == test['composer']).astype(np.int8)

In [199]:
train['artist_composer'].value_counts()

0    7154274
1     223144
Name: artist_composer, dtype: int64

### artist_composer_lyricist: whether the same

In [200]:
train['artist_composer_lyricist'] = ((train['artist_name'] == train['composer']) & (train['artist_name'] == train['lyricist']) & (train['composer'] == train['lyricist'])).astype(np.int8)
test['artist_composer_lyricist'] = ((test['artist_name'] == test['composer']) & (test['artist_name'] == test['lyricist']) & (test['composer'] == test['lyricist'])).astype(np.int8)

In [201]:
train['artist_composer_lyricist'].value_counts()

0    7251449
1     125969
Name: artist_composer_lyricist, dtype: int64

In [294]:
train.to_csv("train_week3_newest_2.csv", index = False)

In [295]:
test.to_csv("test_week3_newest_2.csv", index = False)

In [296]:
train.shape

(7377418, 39)